In [31]:
import os
import google.generativeai as genai
import sys
sys.path.append('../..')
import utils

import panel as pn  # GUI
pn.extension()

In [32]:
# Set up Gemini API key
genai.configure(api_key="")  # Replace with your actual API key

In [33]:
import google.generativeai as genai

models = genai.list_models()
print([m.name for m in models])

['models/chat-bison-001', 'models/text-bison-001', 'models/embedding-gecko-001', 'models/gemini-1.0-pro-vision-latest', 'models/gemini-pro-vision', 'models/gemini-1.5-pro-latest', 'models/gemini-1.5-pro-001', 'models/gemini-1.5-pro-002', 'models/gemini-1.5-pro', 'models/gemini-1.5-flash-latest', 'models/gemini-1.5-flash-001', 'models/gemini-1.5-flash-001-tuning', 'models/gemini-1.5-flash', 'models/gemini-1.5-flash-002', 'models/gemini-1.5-flash-8b', 'models/gemini-1.5-flash-8b-001', 'models/gemini-1.5-flash-8b-latest', 'models/gemini-1.5-flash-8b-exp-0827', 'models/gemini-1.5-flash-8b-exp-0924', 'models/gemini-2.5-pro-exp-03-25', 'models/gemini-2.5-pro-preview-03-25', 'models/gemini-2.5-flash-preview-04-17', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-exp-image-generation', 'models/gemini-2.0-flash-lite-001', 'models/gemini-2.0-flash-lite', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-flash-lite-p

In [34]:
def get_completion_from_messages(messages, model="gemini-1.5-pro-latest", temperature=0.7, max_tokens=500):
    genai.configure(api_key="")  # Replace with your actual API key

    client = genai.GenerativeModel(model_name=model)  # Initialize Gemini model

    # Remove "system" role and merge instructions into user's first message
    formatted_messages = []
    system_message = ""

    for message in messages:
        if message["role"] == "system":
            system_message = message["content"]  # Store system instruction
        elif "content" in message:  
            formatted_messages.append({
                "role": message["role"],  
                "parts": [{"text": message["content"]}]  # Correct format for Gemini
            })
        else:
            raise ValueError(f"Message missing 'content' field: {message}")  # Debugging

    # If there was a system message, merge it into the first user message
    if formatted_messages and system_message:
        formatted_messages[0]["parts"][0]["text"] = system_message + "\n\n" + formatted_messages[0]["parts"][0]["text"]

    response = client.generate_content(
        formatted_messages,  
        generation_config={
            "temperature": temperature,
            "max_output_tokens": max_tokens
        }
    )

    return response.text

In [35]:
# Function to process user messages
def process_user_message(user_input, all_messages, debug=True):
    import utils  # Ensure utils.py is available and correct

    delimiter = "```"

    # Step 1: Input validation
    if debug:
        print("Step 1: Input passed basic check.")

    # Step 2: Extract products and categories
    category_and_product_response = utils.find_category_and_product_only(
        user_input, utils.get_products_and_category()
    )
    print(category_and_product_response)
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    print(category_and_product_list)

    if debug:
        print(f"Step 2: Extracted products: {category_and_product_list}")

    # Step 3: Fetch product information
    product_information = utils.generate_output_string(category_and_product_list)

    if debug:
        print("Step 3: Retrieved product information.")

    # Step 4: Generate customer service response
    step_4_system_message_content = "You are a helpful AI assistant. Provide detailed and accurate information."

    messages = [
        {
            "role": "user",
            "content": step_4_system_message_content + f"\n\n{delimiter}{user_input}{delimiter}",
        },
        {"role": "assistant", "content": f"Relevant product information:\n{product_information}"},
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    all_messages.append({"role": "assistant", "content": final_response})

    if debug:
        print("Step 4: Generated response.")

    # Step 5: Evaluate response quality

    # Step 6: Evaluate response quality
    step_6_system_message_content = "Evaluate whether the AI response fully answers the customer's question. Reply with only 'Y' or 'N'."

    evaluation_messages = [
        {
            "role": "user",
            "content": step_6_system_message_content
            + f"\n\nCustomer message: {delimiter}{user_input}{delimiter}\n"
            + f"Agent response: {delimiter}{final_response}{delimiter}\n"
            + f"Does the response sufficiently answer the question? Reply only with Y or N.",
        }
    ]

    evaluation_response = get_completion_from_messages(evaluation_messages)

    # Debugging output
    if debug:
        print("Raw evaluation response:", evaluation_response)

    # Ensure cleaned and flexible check
    cleaned_evaluation_response = evaluation_response.strip().upper()

    # Step 7: Decision based on evaluation
    if cleaned_evaluation_response.startswith("Y"):
        if debug:
            print("Step 7: Response is approved.")
        return final_response, all_messages
    else:
        if debug:
            print("Step 7: Response is not sufficient.")
        return "I'm unable to provide the information you're looking for. Let me connect you with a representative.", all_messages

user_input = "tell me about the SmartX Pro Phone and the FotoSnap Camera, the DSLR one. Also, what tell me about your TVs?"
response, _ = process_user_message(user_input, [])
print(response)


Step 1: Input passed basic check.
```json
[
  {
    "category": "Smartphones and Accessories",
    "products": [
      "SmartX ProPhone"
    ]
  },
  {
    "category": "Cameras and Camcorders",
    "products": [
      "FotoSnap DSLR Camera"
    ]
  },
  {
    "category": "Televisions and Home Theater Systems",
    "products": [
      "CineView 4K TV",
      "SoundMax Home Theater",
      "CineView 8K TV",
      "SoundMax Soundbar",
      "CineView OLED TV"
    ]
  }
]
```
[{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]
Step 2: Extracted products: [{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category'

In [36]:
pn.extension(raw_css=['''
.assistant-response {
    background-color: #F6F6F6;
    padding: 10px;
    border-radius: 5px;
}
'''])

In [37]:
def collect_messages(debug=False):
    user_input = inp.value
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context

    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role': 'assistant', 'content': f"{response}"})

    panels.append(pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600, css_classes=['assistant-response'])))

    return pn.Column(*panels)

In [38]:
panels = []  # collect display
context = [{'role': 'system', 'content': "You are a Service Assistant"}]

inp = pn.widgets.TextInput(placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Traceback (most recent call last):
 File "C:\Users\Admin\anaconda3\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\panel\viewable.py", line 472, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 104, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 443, in invoke_with_curdoc
 return f()
 ^^^
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 104, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\document\document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_event(self, event, setter)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\document\events.py", line 245, in handle_event
 event_cls._handle_event(doc, event)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\document\events.py", line 280, in _handle_event
 cb(event.msg_data)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 390, in trigger_event
 model._trigger_event(event)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\util\callback_manager.py", line 113, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 260, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 443, in invoke_with_curdoc
 return f()
 ^^^
 File "C:\Users\Admin\anaconda3\Lib\site-packages\bokeh\util\callback_manager.py", line 109, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\panel\reactive.py", line 481, in _comm_event
 state._handle_exception(e)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\panel\io\state.py", line 442, in _handle_exception
 raise exception
 File "C:\Users\Admin\anaconda3\Lib\site-packages\panel\reactive.py", line 479, in _comm_event
 self._process_bokeh_event(doc, event)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\panel\reactive.py", line 416, in _process_bokeh_event
 self._process_event(event)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\panel\widgets\button.py", line 246, in _process_event
 self.param.trigger('value')
 File "C:\Users\Admin\anaconda3\Lib\site-packages\param\parameterized.py", line 2440, in trigger
 self_.update(dict(params, **triggers))
 File "C:\Users\Admin\anaconda3\Lib\site-packages\param\parameterized.py", line 2278, in update
 restore = self_._update(*args, **kwargs)
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Admin\anaconda3\Lib\site-packages\param\parameterized.py", line 2318, in _update
 self_._batch_call_watchers()
 File "C:\Users\Admin\anaconda3\Lib\site-packages\param\parameterized.py", line 2509, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\param\parameterized.py", line 2471, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\Admin\anaconda3\Lib\site-packages\panel\param.py", line 865, in _replace_pane
 new_object = self.eval(self.object)
 ^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Admin\anaconda3\Lib\site-packages\panel\param.py", line 824, in eval
 return eval_function_with_deps(function)
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Admin\anaconda3\Lib\site-packages\param\parameterized.py", line 162, in eval_function_with_deps
 return function(*args, **kwargs)
 ^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Admin\anaconda3\Lib\site-packages\param\depends.py", line 41, in _depends
 r

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)